In [48]:
%matplotlib inline
from matplotlib.pyplot import figure, show
import matplotlib.pyplot as plt
import json
import csv
import pandas as pd
import sklearn.feature_extraction.text as sk_text
import io
import requests
import numpy as np
from scipy.stats import zscore
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn import metrics

#------------------PROFESSOR'S CODE, HIDDEN GEM--------------------------
def encode_numeric_range(df, name, normalized_low=-1, normalized_high=1,
                         data_low=None, data_high=None):
    if data_low is None:
        data_low = min(df[name])
        data_high = max(df[name])

    df[name] = ((df[name] - data_low) / (data_high - data_low)) * (normalized_high - normalized_low) + normalized_low

#fix headers for census data
def fix_table(df,dropig=True):
    new_header = df.iloc[0]
    df = df[1:]
    df.columns = new_header
    df = df.astype({"Id2" : int})
    if dropig:
        df = df.drop(['Id','Geography'],axis=1)
    return df


In [51]:
dp1  = pd.read_csv('census_data/DEC_00_110H_DP1_with_ann.csv', encoding="utf-8")
dp2  = pd.read_csv('census_data/DEC_00_110S_DP2_with_ann.csv', encoding="utf-8")
dp3  = pd.read_csv('census_data/DEC_00_110S_DP3_with_ann.csv', encoding="utf-8")
dp4  = pd.read_csv('census_data/DEC_00_110S_DP4_with_ann.csv', encoding="utf-8")
h002 = pd.read_csv('census_data/DEC_00_110H_H002_with_ann.csv', encoding="utf-8")

#fix headers
dp1  = fix_table(dp1,False)
dp2  = fix_table(dp2)
dp3  = fix_table(dp3)
dp4  = fix_table(dp4)
h002 = fix_table(h002)

df = dp1.copy()
df = df.join(dp2,on="Id2", rsuffix="_dp2")
df = df.join(dp3,on="Id2", rsuffix="_dp3")
df = df.join(dp4,on="Id2", rsuffix="_dp4")
df = df.join(h002,on="Id2", rsuffix="_h002")

df

,Id,Id2,Geography,Number; Total population,Percent; Total population,Number; Total population - SEX AND AGE - Male,Percent; Total population - SEX AND AGE - Male,Number; Total population - SEX AND AGE - Female,Percent; Total population - SEX AND AGE - Female,Number; Total population - SEX AND AGE - Under 5 years,...,Percent; Specified renter-occupied units - GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN 1999 - 35 percent or more,Number; Specified renter-occupied units - GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN 1999 - Not computed,Percent; Specified renter-occupied units - GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN 1999 - Not computed,Id2_h002,Total:,Urban:,Urban: - Inside urbanized areas,Urban: - Inside urban clusters,Rural,Not defined for this file
1,5001000US0101,101,"Congressional District 1 (110th Congress), Ala...",635300,100.0,306620,48.3,328680,51.7,44487,...,30.1,4645,7.3,1214.0,375780,340665,313304,27361,35115,0
2,5001000US0102,102,"Congressional District 2 (110th Congress), Ala...",635300,100.0,308305,48.5,326995,51.5,42135,...,31.9,5407,7.7,1215.0,305220,272298,269652,2646,32922,0
3,5001000US0103,103,"Congressional District 3 (110th Congress), Ala...",635300,100.0,305852,48.1,329448,51.9,40841,...,30.1,4430,9.3,1216.0,316721,267223,217371,49852,49498,0
4,5001000US0104,104,"Congressional District 4 (110th Congress), Ala...",635300,100.0,310279,48.8,325021,51.2,40985,...,38.2,7622,8.4,1217.0,236236,236236,236236,0,0,0
5,5001000US0105,105,"Congressional District 5 (110th Congress), Ala...",635300,100.0,309694,48.7,325606,51.3,41451,...,39.3,9640,7.3,1218.0,300463,294501,242406,52095,5962,0
6,5001000US0106,106,"Congressional District 6 (110th Congress), Ala...",635300,100.0,310240,48.8,325060,51.2,41352,...,33.7,4469,7.5,1219.0,326763,325800,325800,0,963,0
7,5001000US0107,107,"Congressional District 7 (110th Congress), Ala...",635300,100.0,295514,46.5,339786,53.5,44741,...,35.9,5911,7.1,1220.0,318204,316940,316940,0,1264,0
8,5001000US0200,200,Congressional District (at Large) (110th Congr...,626932,100.0,324112,51.7,302820,48.3,47591,...,25.7,5169,10.2,2601.0,390670,102297,1159,101138,288373,0
9,5001000US0401,401,"Congressional District 1 (110th Congress), Ari...",641329,100.0,325515,50.8,315814,49.2,44745,...,27.6,4918,7.1,4901.0,261937,227598,200344,27254,34339,0
10,5001000US0402,402,"Congressional District 2 (110th Congress), Ari...",641329,100.0,313683,48.9,327646,51.1,40887,...,26.8,5120,6.9,4902.0,285074,231125,201439,29686,53949,0
